Movie Recommendation Helper

Flow: User gives a genre → Prompt asks LLM to suggest 3 movies → Next node fetches a short description or rating for each movie → Final output: a neat recommendation list.

In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph , START , END
from typing import TypedDict

In [2]:
model = ChatOpenAI(model='gpt-5-nano')

In [3]:
class Moviestate(TypedDict):
    genre:str
    movies:str
    description:str
    recommended:str 

In [4]:
def movie_suggest(state:Moviestate) -> Moviestate:
    genre=state['genre']
    prompt=f"Suggest some 3 best movies in this genre - {genre}"
    state['movies']=model.invoke(prompt).content
    return state 

def movie_description(state:Moviestate) -> Moviestate:
    movies=state['movies']
    prompt=f"Write description of each of these 3 movies - {movies}"
    state['description']=model.invoke(prompt).content
    return state 

def movie_recommended(state:Moviestate) -> Moviestate:
    genre=state['genre']
    movies=state['movies']
    description=state['description']
    prompt=f"Based on this {genre} and looking at {description} recommend best {movies}"
    state['recommended']=model.invoke(prompt).content
    return state
     

In [5]:
graph=StateGraph(Moviestate)

graph.add_node("movie_suggest",movie_suggest)
graph.add_edge(START,"movie_suggest")
graph.add_node("movie_description",movie_description)
graph.add_edge("movie_suggest","movie_description")
graph.add_node("movie_recommended",movie_recommended)
graph.add_edge("movie_description","movie_recommended")
graph.add_edge("movie_recommended",END)

workflow=graph.compile()

In [6]:
output=workflow.invoke({"genre":"action movies"})
print(output)

{'genre': 'action movies', 'movies': 'Here are three widely regarded top action movies you might enjoy:\n\n1) The Dark Knight (2008) — A gripping superhero/action blend from Christopher Nolan, anchored by Heath Ledger’s iconic Joker and standout set-piece chaos.\n\n2) Terminator 2: Judgment Day (1991) — Groundbreaking, high-octane sci-fi action with groundbreaking practical effects and a compelling human story.\n\n3) Mad Max: Fury Road (2015) — Non-stop, visually stunning chase movie with practical effects, minimal dialogue, and adrenaline-fueled sequences.\n\nWant options in a different style (martial-arts focus, spy thrillers, newer releases, or pure gun-fu)? I can tailor a few more picks.', 'description': 'Here are concise descriptions for each film:\n\n- The Dark Knight (2008)\n  Christopher Nolan blends superhero spectacle with dark crime thriller elements. The film threads Batman’s vigilante mission against Gotham’s criminal underworld with a chillingly chaotic villain in the Jok

In [7]:
print(output["genre"])

action movies


In [8]:
print(output["movies"])

Here are three widely regarded top action movies you might enjoy:

1) The Dark Knight (2008) — A gripping superhero/action blend from Christopher Nolan, anchored by Heath Ledger’s iconic Joker and standout set-piece chaos.

2) Terminator 2: Judgment Day (1991) — Groundbreaking, high-octane sci-fi action with groundbreaking practical effects and a compelling human story.

3) Mad Max: Fury Road (2015) — Non-stop, visually stunning chase movie with practical effects, minimal dialogue, and adrenaline-fueled sequences.

Want options in a different style (martial-arts focus, spy thrillers, newer releases, or pure gun-fu)? I can tailor a few more picks.


In [9]:
print(output["description"])

Here are concise descriptions for each film:

- The Dark Knight (2008)
  Christopher Nolan blends superhero spectacle with dark crime thriller elements. The film threads Batman’s vigilante mission against Gotham’s criminal underworld with a chillingly chaotic villain in the Joker, creating a moral maze about justice, power, and sacrifice. Heath Ledger’s Joker delivers an unforgettable, iconic performance that fuels some of the film’s most intense set pieces and philosophical riffs, all anchored by strong performances, practical stunt work, and gripping IMAX sequences.

- Terminator 2: Judgment Day (1991)
  A groundbreaking action-sci-fi experience that pairs heart-pounding chase sequences with a surprisingly human core. The T-800 returns as a protector for John Connor, while a relentlessly threatening T-1000 pushes the visuals and effects to new heights. With a mix of practical effects, innovative makeup, and thrilling set pieces, the film combines big-budget spectacle with a story abo

In [10]:
print(output["recommended"])

Nice concise set. Here’s a polished version of your descriptions, plus a few tailored picks you can use depending on style.

Refined descriptions
- The Dark Knight (2008)
  A gripping blend of superhero spectacle and crime thriller. Batman’s battle against Gotham’s underworld collides with the chaotic genius of the Joker, creating a moral puzzle about justice, power, and sacrifice. Heath Ledger’s Joker anchors some of the film’s most memorable set pieces and philosophical moments, supported by strong performances, practical stunts, and gripping IMAX sequences.

- Terminator 2: Judgment Day (1991)
  An action-sci-fi landmark that pairs high-octane chases with a surprisingly human core. The T-800 protects John Connor as the T-1000 pushes the limits of practical effects and makeup. The film fuses big-budget spectacle with a story about loyalty, redemption, and the costs of saving the future.

- Mad Max: Fury Road (2015)
  A relentless desert chase driven by practical effects and spectacul